# Introduction à Pytorch

## Installation et Import

In [1]:
# Si PyTorch n'est pas installé, décommente la ligne suivante :
# !pip install torch torchvision

import torch

print('PyTorch version:', torch.__version__)

PyTorch version: 2.5.1+cu121


## 1.Les Tenseurs

PyTorch définit une classe appelée Tensor (torch. Tensor) pour stocker les données et opérer sur des tableaux rectangulaires multidimensionnels homogènes de nombres. Ils sont similaires aux tableaux NumPy, mais permettent entre autres l'exécution sur GPU et le calcul automatique des gradients.

### 1.1 Création de Tenseurs

In [2]:
# Création d'un tenseur à partir d'une liste
a = torch.tensor([1.0, 2.0, 3.0])
print('Tenseur a:', a)
print('Dimensions de a:', a.shape)
print('Type de données de a:', a.dtype)

Tenseur a: tensor([1., 2., 3.])
Dimensions de a: torch.Size([3])
Type de données de a: torch.float32


### 1.2  Exemples

In [3]:
# Tenseur rempli de zéros
z = torch.zeros((2, 3))
print('Tenseur de zéros:', z)

# Tenseur aléatoire
r = torch.rand((2, 2))
print('Tenseur aléatoire:', r)

Tenseur de zéros: tensor([[0., 0., 0.],
        [0., 0., 0.]])
Tenseur aléatoire: tensor([[0.0014, 0.1866],
        [0.5420, 0.3643]])


## 2. Opérations sur les tenseurs 

In [4]:
a = torch.tensor([1.0, 2.0])
b = torch.tensor([3.0, 4.0])

# Addition
addition = a + b
print('Addition:', addition)

# Multiplication élément par élément
mult_elem = a * b
print('Multiplication élément par élément:', mult_elem)

# Produit scalaire
dot_product = torch.dot(a, b)
print('Produit scalaire:', dot_product)

Addition: tensor([4., 6.])
Multiplication élément par élément: tensor([3., 8.])
Produit scalaire: tensor(11.)


# 4. Autograd

Le module autograd offre un moyen simple de calculer automatiquement les gradients, utilisés pour optimiser les paramètres du modèle via la descente de gradient, pour toute fonction opérée au sein des réseaux neuronaux. L'ajout d'un tenseur avec requires_grad=True indique à autograd que chaque opération sur ce tenseur doit être suivie, ce qui permet une différenciation automatique.

In [5]:
# Création d'un tenseur pour lequel on souhaite calculer le gradient
x = torch.tensor([2.0], requires_grad=True)

# Définition d'une fonction : y = x^2 + 3x + 1
y = x**2 + 3*x + 1

# Calcul du gradient dy/dx
y.backward()

print('Gradient de y par rapport à x:', x.grad)

Gradient de y par rapport à x: tensor([7.])


# 5. Régression linaire
Nous allons créer un modèle de régression linéaire qui apprend la relation suivante : y = 2x.

In [6]:
# Données d'entrée et sortie
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]])

# Initialisation des paramètres (poids et biais) avec gradient activé
w = torch.tensor([[0.0]], requires_grad=True)
b = torch.tensor([[0.0]], requires_grad=True)

# Taux d'apprentissage
lr = 0.01

# Boucle d'entraînement
for epoch in range(100):
    # Prédiction : calcul de la sortie du modèle
    y_pred = x @ w + b

    # Calcul de l'erreur (Mean Squared Error)
    loss = ((y_pred - y) ** 2).mean()

    # Calcul des gradients via backpropagation
    loss.backward()

    # Mise à jour des paramètres sans traçabilité (désactivation temporaire du calcul automatique)
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad

        # Remise à zéro des gradients afin qu'ils ne s'accumulent pas
        w.grad.zero_()
        b.grad.zero_()

    # Affichage périodique de la perte
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item()}')

print(f'Paramètres finaux: w = {w.item()}, b = {b.item()}')


Epoch 0: Loss = 30.0
Epoch 10: Loss = 0.8330377340316772
Epoch 20: Loss = 0.07510896027088165
Epoch 30: Loss = 0.052382320165634155
Epoch 40: Loss = 0.048858530819416046
Epoch 50: Loss = 0.04600245878100395
Epoch 60: Loss = 0.04332456737756729
Epoch 70: Loss = 0.04080278053879738
Epoch 80: Loss = 0.03842790424823761
Epoch 90: Loss = 0.0361911877989769
Paramètres finaux: w = 1.846347451210022, b = 0.45175737142562866


Nous allons initialiser les paramètres (poids et biais) et utiliser la descente de gradient pour ajuster ces paramètres.Le résultat final devrait s'approcher d'une valeur de w égale à 2 et d'un b proche de 0.


# 6. Utilisation de PyTorch pour la Génération d'Images

PyTorch est particulièrement populaire pour entraîner des modèles de génération d’images. Par exemple, grâce à autograd, le calcul des gradients est automatique, simplifiant ainsi l'entraînement des réseaux de neurones. De plus, PyTorch permet d'exécuter des calculs sur GPU, accélérant considérablement l'entraînement, ce qui est particulièrement utile pour les grands modèles de génération d'images.

## 6.1 Exemple simplifié

In [7]:
import torch.nn as nn
import torch.optim as optim

# Exemple d'un générateur simplifié
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 784),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(z.size(0), 1, 28, 28)  # Reshape pour obtenir une image 28x28 (exemple MNIST)
        return img

# Création du générateur
netG = Generator()

# Optimiseur
optimizerG = optim.Adam(netG.parameters(), lr=0.0002)

# Exemple d'entraînement pour un batch fictif
batch_size = 64
noise = torch.randn(batch_size, 100)  # Bruit aléatoire
fake_images = netG(noise)

# Affichage de la forme du batch d'images générées
print('Shape des images générées:', fake_images.shape)


Shape des images générées: torch.Size([64, 1, 28, 28])
